In [ ]:
%reset -f

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import time
import statistics
from tqdm import tqdm

from MINE.Log import Log
from MINE.Analysis import SessionAnalytics, ExperimentAnalytics
from MINE.StreamFilter import IStreamFilter, TimestampStreamFilter
from MINE.SessionFilters import ISessionFilter, ContainsStreamSessionFilter, ContainsMarkersSessionFilter
from MINE.StreamProcessing import StreamProcesses
from MINE.StreamOutput import StreamOutput

In [ ]:
class ProjectData:
    def __init__(self):
        self.video_data_dictionary: dict[str, VideoEntry] = {}
        pass

class VideoEntry:
    def __init__(self):
        #Video Meta-Data
        self.video_id: str | None = None
        self.full_duration: float | None = None
        self.video_duration: float | None = None

        #Biophysiological Data
        self.gyroscope_data: list[pd.DataFrame] = []
        self.accelerometer_data: list[pd.DataFrame] = []

        #Heart Rate Data
        self.baseline_heart_rates: list[float] = []
        self.video_mean_heart_rates: list[float] = []
        self.heart_rate: list[pd.DataFrame] = []
        self.heart_rate_baseline_deviation: list[pd.DataFrame] = []
        pass

In [ ]:
def stream_filter() -> ContainsStreamSessionFilter:
    return ContainsStreamSessionFilter([
        "Marker",
        "PPG_GRN",
        "PPG_RED",
        "PPG_IR",
    ])

def marker_filter() -> ContainsMarkersSessionFilter:
    return ContainsMarkersSessionFilter("Marker", [
        "Video: Be a floater, Event: VideoStart",
        "Video: Be a floater, Event: VideoEnd",
        "Video: RNLI Respect the water “Ladbible short film”, Event: VideoStart",
        "Video: RNLI Respect the water “Ladbible short film”, Event: VideoEnd",
        "Video: “Evans story”, Event: VideoStart",
        "Video: “Evans story”, Event: VideoEnd",
        "Video: “Little girl being rescued by RNLI”, Event: VideoStart",
        "Video: “Little girl being rescued by RNLI”, Event: VideoEnd",
        "Video: Alfie’s phone, Event: VideoStart",
        "Video: Alfie’s phone, Event: VideoEnd",
        "Video: “Float to Live”, Event: VideoStart",
        "Video: “Float to Live”, Event: VideoEnd",
        "Video: Respect the Water via the NWSF\u202f‘make the right call’, Event: VideoStart",
        "Video: Respect the Water via the NWSF\u202f‘make the right call’, Event: VideoEnd",
        "Video: “Seaside safety song”, Event: VideoStart",
        "Video: “Seaside safety song”, Event: VideoEnd",
        "Video: RNLI the breath test, Event: VideoStart",
        "Video: RNLI the breath test, Event: VideoEnd",
        "Video: RNLI Christmas bed-time story, Event: VideoStart",
        "Video: RNLI Christmas bed-time story, Event: VideoEnd",
        "Video: “RNLI: The heart-breaking story of Liam Hall”, Event: VideoStart",
        "Video: “RNLI: The heart-breaking story of Liam Hall”, Event: VideoEnd",
    ])

def participant_data() -> pd.DataFrame:
    return pd.DataFrame(columns=["Participant_ID", "File_Path"], data=[
        ["P01 - 535679", "V:/Data/Analysis/RNLI/participant 1.xdf"],
        ["P02 - 888462", "V:/Data/Analysis/RNLI/participant 2.xdf"],
        ["P03 - 499031", "V:/Data/Analysis/RNLI/participant 3.xdf"],
        ["P04.0 - 832362", "V:/Data/Analysis/RNLI/participant 4.0.xdf"],
        ["P04.1 - 832362", "V:/Data/Analysis/RNLI/participant 4.1.xdf"],
        ["P05", "V:/Data/Analysis/RNLI/participant 5.xdf"],
        ["P06", "V:/Data/Analysis/RNLI/participant 6.xdf"],
        ["P07", "V:/Data/Analysis/RNLI/participant 7.xdf"],
        ["P08", "V:/Data/Analysis/RNLI/participant 8.xdf"],
        ["P09", "V:/Data/Analysis/RNLI/participant 9.xdf"],
        ["P10 - 999149", "V:/Data/Analysis/RNLI/participant 10.xdf"],
        ["P11 - 153327", "V:/Data/Analysis/RNLI/participant 11.xdf"],
        ["P12", "V:/Data/Analysis/RNLI/participant 12.xdf"],
        ["P13", "V:/Data/Analysis/RNLI/participant 13.xdf"],
        ["P14", "V:/Data/Analysis/RNLI/participant 14.xdf"],
        ["P15 - 156103", "V:/Data/Analysis/RNLI/participant 15.xdf"],
        ["P16 - 701399", "V:/Data/Analysis/RNLI/participant 16.xdf"]
    ])

In [ ]:
_project_data: ProjectData = ProjectData()

_experiment_analytics: ExperimentAnalytics = ExperimentAnalytics.create_from_paths(participant_data())
_experiment_analytics: ExperimentAnalytics = _experiment_analytics.get_filtered_subset([
    marker_filter(),
    stream_filter()
])

In [ ]:
def retrieve_gyroscope_data(video_analytics: SessionAnalytics, video_entry: VideoEntry):
    gyroscope_x: pd.DataFrame = video_analytics.stream_data_dictionary["GYRO_X"]
    gyroscope_y: pd.DataFrame = video_analytics.stream_data_dictionary["GYRO_Y"]
    gyroscope_z: pd.DataFrame = video_analytics.stream_data_dictionary["GYRO_Z"]

    x_timestamps: np.ndarray = gyroscope_x['Timestamp'].to_numpy()
    y_timestamps: np.ndarray = gyroscope_y['Timestamp'].to_numpy()
    z_timestamps: np.ndarray = gyroscope_z['Timestamp'].to_numpy()

    x_values: np.ndarray = np.concatenate(gyroscope_x['Value'].to_numpy())
    y_values: np.ndarray = np.concatenate(gyroscope_y['Value'].to_numpy())
    z_values: np.ndarray = np.concatenate(gyroscope_z['Value'].to_numpy())

    idx_y: np.ndarray = np.abs(y_timestamps[:, None] - x_timestamps).argmin(axis=0)
    idx_z: np.ndarray = np.abs(z_timestamps[:, None] - x_timestamps).argmin(axis=0)

    aligned_y_values: np.ndarray = y_values[idx_y]
    aligned_z_values: np.ndarray = z_values[idx_z]

    magnitudes: np.ndarray = np.sqrt(np.square(x_values) + np.square(aligned_y_values) + np.square(aligned_z_values))
    video_entry.gyroscope_data.append(pd.DataFrame({
        "Value": magnitudes,
        "Timestamp": x_timestamps
    }))

def retrieve_accelerometer_data(video_analytics: SessionAnalytics, video_entry: VideoEntry):
    accelerometer_x: pd.DataFrame = video_analytics.stream_data_dictionary["ACC_X"]
    accelerometer_y: pd.DataFrame = video_analytics.stream_data_dictionary["ACC_Y"]
    accelerometer_z: pd.DataFrame = video_analytics.stream_data_dictionary["ACC_Z"]

    x_timestamps: np.ndarray = accelerometer_x['Timestamp'].to_numpy()
    y_timestamps: np.ndarray = accelerometer_y['Timestamp'].to_numpy()
    z_timestamps: np.ndarray = accelerometer_z['Timestamp'].to_numpy()

    x_values: np.ndarray = np.concatenate(accelerometer_x['Value'].to_numpy())
    y_values: np.ndarray = np.concatenate(accelerometer_y['Value'].to_numpy())
    z_values: np.ndarray = np.concatenate(accelerometer_z['Value'].to_numpy())

    idx_y: np.ndarray = np.abs(y_timestamps[:, None] - x_timestamps).argmin(axis=0)
    idx_z: np.ndarray = np.abs(z_timestamps[:, None] - x_timestamps).argmin(axis=0)

    aligned_y_values: np.ndarray = y_values[idx_y]
    aligned_z_values: np.ndarray = z_values[idx_z]

    magnitudes: np.ndarray = np.sqrt(np.square(x_values) + np.square(aligned_y_values) + np.square(aligned_z_values))
    video_entry.accelerometer_data.append(pd.DataFrame({
        "Value": magnitudes,
        "Timestamp": x_timestamps
    }))

def retrieve_heart_rate_data(video_analytics: SessionAnalytics, video_entry: VideoEntry, sample_duration: float = 20, sampling_step_count: float = 5):
    systolic_peaks = video_analytics.stream_data_dictionary["PPG_GRN_Filtered_Systolic_Peaks"]
    bpm_dataframe = pd.DataFrame(columns=["Timestamp", "BPM"])

    #region [ Calculate Average Heartrate ]
    min_bpm: float = float("inf")
    max_bpm: float = 0

    sample_time = -60 + (sample_duration / 2)
    while sample_time < (video_entry.video_duration - (sample_duration / 2)):
        peaks_subset = systolic_peaks[
            (systolic_peaks["Timestamp"] > (sample_time - (sample_duration / 2))) &
            (systolic_peaks["Timestamp"] < (sample_time + (sample_duration / 2)))
        ]

        bpm = len(peaks_subset) * (60 / sample_duration)

        if bpm < min_bpm: min_bpm = bpm
        if bpm > max_bpm: max_bpm = bpm

        bpm_dataframe.loc[len(bpm_dataframe)] = [
            sample_time,
            bpm
        ]

        sample_time = sample_time + sampling_step_count
    #endregion



    #region [ Calculate Baseline ]
    baseline_sample_subset = bpm_dataframe[
        (bpm_dataframe["Timestamp"] > -30) &
        (bpm_dataframe["Timestamp"] < 0)
    ]

    baseline_heartrate = baseline_sample_subset["BPM"].mean()
    #endregion

    #region [ Calculate Average Video Heartrate ]
    video_mean_sample_subset = bpm_dataframe[
        (bpm_dataframe["Timestamp"] > 0) &
        (bpm_dataframe["Timestamp"] < video_entry.video_duration)
    ]

    video_mean_heartrate = video_mean_sample_subset["BPM"].mean()
    #endregion

    #region [ Calculate Heartrate Baseline Deviation ]
    baseline_deviation_dataframe = pd.DataFrame({
        "Timestamp": bpm_dataframe["Timestamp"],
        "Deviation": pd.Series(bpm_dataframe["BPM"]).apply(lambda value: value / baseline_heartrate)
    })
    #endregion

    video_entry.baseline_heart_rates.append(baseline_heartrate)
    video_entry.video_mean_heart_rates.append(video_mean_heartrate)

    video_entry.heart_rate.append(bpm_dataframe)
    video_entry.heart_rate_baseline_deviation.append(baseline_deviation_dataframe)


def generate_video_dictionary_entries(session_analytics: SessionAnalytics):
    #region [ Create Video List For Session ]
    marker_pairs = session_analytics.get_paired_markers("Marker", "VideoStart", "VideoEnd")

    marker_pairs = marker_pairs[~marker_pairs['Start Marker'].str.contains('panda', case=False, na=False)]
    marker_pairs = marker_pairs[~marker_pairs['Start Marker'].str.contains('spiders', case=False, na=False)]
    #endregion

    gyro_durations = []
    acc_durations = []
    heart_rate_durations = []

    for video_index, row in marker_pairs.iterrows():
        #region [ Get Video Data ]
        video_starting_marker = row["Start Marker"]
        video_start_time = row["Start Timestamp"]
        video_end_time = row["End Timestamp"]
        previous_calibration_start_time = video_start_time - 60
        next_calibration_end_time = video_end_time + 60

        video_analytics = session_analytics.get_filtered_subset([TimestampStreamFilter(previous_calibration_start_time, next_calibration_end_time)])
        video_analytics.localise_timestamps(video_start_time)

        video_end_time = video_end_time - video_start_time
        previous_calibration_start_time = previous_calibration_start_time - video_start_time
        next_calibration_end_time = next_calibration_end_time - video_start_time
        video_start_time = 0

        full_duration = video_end_time - previous_calibration_start_time
        video_duration = video_end_time - video_start_time
        #endregion

        #region [ Add Video Data to Dataframe ]
        video_data_dictionary = _project_data.video_data_dictionary

        if video_starting_marker not in video_data_dictionary: video_data_dictionary[video_starting_marker] = VideoEntry()
        video_entry = video_data_dictionary[video_starting_marker]
        video_entry.video_id = video_starting_marker
        video_entry.video_duration = video_duration
        video_entry.full_duration = full_duration
        #endregion

        #region [ Retrieve Session Data ]
        stopwatch_start_time = time.time()
        retrieve_heart_rate_data(video_analytics, video_entry)
        stopwatch_end_time = time.time()
        heart_rate_durations.append(stopwatch_end_time - stopwatch_start_time)

        stopwatch_start_time = time.time()
        retrieve_gyroscope_data(video_analytics, video_entry)
        stopwatch_end_time = time.time()
        gyro_durations.append(stopwatch_end_time - stopwatch_start_time)

        stopwatch_start_time = time.time()
        retrieve_accelerometer_data(video_analytics, video_entry)
        stopwatch_end_time = time.time()
        acc_durations.append(stopwatch_end_time - stopwatch_start_time)
        #endregion



In [ ]:
def generate_global_dictionaries():
    for _, row in tqdm(_experiment_analytics.analytics_dataframe.iterrows(), total=_experiment_analytics.analytics_dataframe.shape[0], desc="Processing Sessions"):
        session_analytics: SessionAnalytics = row["Analysis_Object"]
        session_analytics.localise_timestamps()

        StreamProcesses.butterworth_filter(session_analytics, "PPG_GRN", "PPG_GRN_Filtered")
        StreamProcesses.detect_ppg_peaks(session_analytics, "PPG_GRN_Filtered")

        generate_video_dictionary_entries(session_analytics)

generate_global_dictionaries()

In [ ]:
def generate_global_figure():
    video_data_dictionary = _project_data.video_data_dictionary

    video_index = 0
    figure_rows = 5

    for key, value in tqdm(video_data_dictionary.items(), total=len(video_data_dictionary), desc="Generating Video Figures"):
        video_entry: VideoEntry = value
        video_index += 1

        figure, axes = plt.subplots(nrows=figure_rows, ncols=1, figsize=(video_entry.full_duration * 0.5, figure_rows * 4), dpi=300)

        for _, entry in enumerate(video_entry.gyroscope_data): axes[0].plot(entry["Timestamp"], entry["Value"])
        for _, entry in enumerate(video_entry.accelerometer_data): axes[1].plot(entry["Timestamp"], entry["Value"])
        for _, entry in enumerate(video_entry.heart_rate): axes[2].plot(entry["Timestamp"], entry["BPM"])
        for _, entry in enumerate(video_entry.heart_rate_baseline_deviation): axes[3].plot(entry["Timestamp"], entry["Deviation"])

        for _, axis in enumerate(axes): axis.vlines(0, axis.get_ylim()[0], axis.get_ylim()[1], linestyles='dashed', alpha=0.5, colors= "black")
        for _, axis in enumerate(axes): axis.set_xlim(-60, video_entry.video_duration)

        os.makedirs(f"V:/Exports/Global Analysis", exist_ok=True)
        plt.savefig(f"V:/Exports/Global Analysis/{video_index} Video Data.png")
        plt.close(figure)

generate_global_figure()

In [ ]:
def view_data():
    pass